In [ ]:

import os

USE_DECOY = os.getenv("FDE_PUBLIC_MODE", "0") == "1"

if USE_DECOY:
    from tiny_universe.toy_engine import TinyEngine
    print(">>> Running Tiny Decoy Universe (Public Mode)")
    engine = TinyEngine()
else:
    from engine import FDEEngine
    print(">>> Running Full FDE Engine (Private Mode)")



In [ ]:
import sys
import importlib

sys.path.append(r"C:\FDE PROJECTS")

In [14]:
import importlib
import fde.personas.guardian
import fde.personas.alpha_multi
import fde.personas.convexity

importlib.reload(fde.personas.guardian)
importlib.reload(fde.personas.alpha_multi)
importlib.reload(fde.personas.convexity)

from fde.personas.guardian import GuardianPersona
from fde.personas.alpha_multi import MultiFactorAlphaPersona
from fde.personas.convexity import ConvexityPersona

from router import PersonaRouter
from engine import FDEEngine


In [15]:


root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root not in sys.path:
    sys.path.append(root)

print("cwd:", os.getcwd())
print("root:", root)

# %%
import pandas as pd
import numpy as np

from fde.factors.cross_section import CrossSectionFactors
from fde.personas.alpha_multi import MultiFactorAlphaPersona
from fde.personas.convexity import ConvexityPersona
from fde.personas.guardian import GuardianPersona
from fde.interfaces.core import (
    MarketSnapshot, PortfolioState, PersonaContext, OptionSnapshot
)
from router import PersonaRouter
from engine import FDEEngine

print("FDE imports OK ✓")


cwd: c:\FDE PROJECTS\notebooks
root: c:\FDE PROJECTS
FDE imports OK ✓


In [16]:
# %%
assets = ["CN_A1", "CN_A2", "CN_BANK", "US_SPY", "US_QQQ", "US_NVDA"]

scores = pd.DataFrame(
    {
        "value":     [0.10, 0.20, 0.15, 0.90, 1.20, 1.50],
        "quality":   [0.30, 0.25, 0.28, 0.85, 1.10, 1.40],
        "momentum": [-0.10, 0.05, 0.00, 1.10, 1.30, 1.60],
    },
    index=assets,
)

factors = CrossSectionFactors(scores=scores)
scores


,value,quality,momentum
CN_A1,0.10,0.30,-0.10
CN_A2,0.20,0.25,0.05
CN_BANK,0.15,0.28,0.00
US_SPY,0.90,0.85,1.10
US_QQQ,1.20,1.10,1.30
US_NVDA,1.50,1.40,1.60


In [17]:

prices = pd.Series([10, 8, 5, 400, 450, 500], index=assets, name="price")

positions = pd.Series(
    [200, 150, 300, -50, -30, -20],
    index=assets,
    name="position",
)

portfolio = PortfolioState(
    positions=positions,
    cash=1_000_000.0,          
    equity=1_000_000.0,        
    pnl=0.0,                   
    meta={"drawdown": -0.12},  
)

ctx = PersonaContext(step=0, mode="demo")


In [18]:
# %%
def universe_gamma_field(s: np.ndarray) -> np.ndarray:
    return np.exp(-(s - 1.0) ** 2 * 10.0) * np.sin(8.0 * s)

S = prices / prices.mean()
gamma_vals = universe_gamma_field(S.values)

greeks = pd.DataFrame(
    {"underlying": assets, "gamma": gamma_vals},
)

options = OptionSnapshot(greeks=greeks)

snapshot = MarketSnapshot(
    timestamp=pd.Timestamp.utcnow(),
    prices=prices,
    volumes=None,
    orderbook=None,
    features=None,
    options=options,
)

snapshot.options.greeks


,underlying,gamma
0,CN_A1,3.656134e-05
1,CN_A2,2.491090e-05
2,CN_BANK,1.216452e-05
3,US_SPY,3.672891e-03
4,US_QQQ,-2.106424e-06
5,US_NVDA,-7.811140e-07


In [19]:
# %%
def dyn_max_lev(ctx: PersonaContext, portfolio: PortfolioState) -> float:
    dd = float(portfolio.meta.get("drawdown", 0.0))
    if dd < -0.20:
        return 0.8
    elif dd < -0.10:
        return 1.0
    else:
        return 1.2

alpha = MultiFactorAlphaPersona(config={"factor_names": ["value", "momentum"]})

convexity = ConvexityPersona(config={"sharpness": 2.0})

guardian = GuardianPersona(config={"max_gross_lev": dyn_max_lev})

router = PersonaRouter(convexity_weight=-0.5)

engine = FDEEngine(
    personas={
        "alpha": alpha,
        "convexity": convexity,
        "guardian": guardian,
    },
    router=router,
)


In [20]:
# %%
portfolio.meta["drawdown"] = -0.12  # 可以改不同值试试

final_signals = engine.step(
    snapshot=snapshot,
    portfolio=portfolio,
    ctx=ctx,
    factors=factors,
)

print("当前 drawdown:", portfolio.meta["drawdown"])
final_signals



===== FDEEngine STEP DEBUG =====
Timestamp: 2025-12-09 22:45:40.630682+00:00
Context mode: demo, step: 0
Num assets (prices): 6

===== PortfolioState DEBUG (ConvexityPersona) =====
Positions:
 CN_A1      200
CN_A2      150
CN_BANK    300
US_SPY     -50
US_QQQ     -30
US_NVDA    -20
Name: position, dtype: int64
Equity: 1000000.0
Meta: {'drawdown': -0.12}


--- Guardian Debug ---
max_gross_lev: 1.0
Gross exposure: 750.0
Gross leverage: 0.00075
Risk weight: 1.0
----------------------


----- Personas Output Summary -----
[alpha] len=6, head:
CN_A1     -0.978788
CN_A2     -0.795265
CN_BANK   -0.868675
US_SPY     0.489394
US_QQQ     0.856440
US_NVDA    1.296894
dtype: float64
--------------------------------
[convexity] len=6, head:
CN_A1     -0.999802
CN_A2     -0.999908
CN_BANK   -0.999978
US_SPY     0.999989
US_QQQ    -0.999999
US_NVDA   -1.000000
Name: abs_gamma, dtype: float64
--------------------------------
[guardian] len=6, head:
CN_A1      1.0
CN_A2      1.0
CN_BANK    1.0
US_SPY 

CN_A1     -0.478887
CN_A2     -0.295311
CN_BANK   -0.368686
US_SPY    -0.010600
US_QQQ     1.356439
US_NVDA    1.796894
dtype: float64